# Jscatter

# Determining PEG matching point for synthesis 

PEG matching composition depends on molecular weight and temperature.

We determine the matching point based on PEG partial specific volume and th density of the needed D2O/H2O scattering length density. 

In [ ]:
%matplotlib widget

import jscatter as js
import numpy as np
js.usempl(True)   # force matplotlib, not needed in Linux

### load specific volume data according to
 Volumetric Study of Aqueous Solutions of Polyethylene Glycol as a Function of the Polymer Molar Mass in the Temperature Range 283.15  to 313.15 K and 0.1 MPa
 Cruz, Raphael da C.;  Martins, Rosana J.;  Cardoso, Márcio J. E. de M.;  Barcia, Oswaldo E. (2009)
 Journal of Solution Chemistry   vol. 38 (8) p. 957-981

In [ ]:
part=js.dL('./partialSpecificVolumePEG.dat')

#### Plot it if you are curious

In [ ]:
if 0:
    p1=js.mplot()
    for pp in part:
        p1.plot(pp._T,pp._vsolute,li=1,legend='Mw=%g ' %pp.Mw)
    p1.Legend()
    p1.Yaxis(label='partial specific volume / ml/g')
    p1.Xaxis(label='T / K')
    p1.Title('PEG desnity')
    p1.Subtitle('T and Mw dependent PEG density from DOI 10.1007/s10953-009-9388-1')

## Set values describing PEG and solvent to calculate
 - Mw :           molecular weight
 - D2O_fractions: D2O/H2O mixing for explicit output 
 - Temperatures : temperatures of the measurements 

In [ ]:
Mw = 2000
D2O_fractions = [1,0.9,0.27]  
Temperatures = [10,20,30]  

In [ ]:
f =np.r_[0:1.05:0.05] # fraction D

contrasts = js.dL()
for D2O_fraction in D2O_fractions:
    for TT in Temperatures:
        T=273.15+TT
      
        # interpolate partial specific volume to Mw , above 10000 it does not change
        densityhPEO=1/part.interpolate(Mw=Mw)[0].interp(T)

        # scale dPEO density from hPEO with addition d mass
        # assuming same specific volume
        densitydPEO = densityhPEO/458.54*500.80
        
        # use correct density to determine coherent SLD (by mixing according to D fraction)
        c=[js.formel.scatteringLengthDensityCalc([f'{1-ff:.2f}C2000H4200O1100',f'{ff:.2f}C2000D4200O1100'],densitydPEO*ff+densityhPEO*(1-ff))[2] for ff in f]
        contrast = js.dA(np.c_[f,c].T)
       
        # scattering length density D2O/H2O mixture
        # get water density and SLD with D2O fraction
        density = js.formel.waterdensity([f'{D2O_fraction:.2f}D2O1',f'{1-D2O_fraction:.2f}H2O1'],T)
        contrast.d2oh2o = js.formel.scatteringLengthDensityCalc([f'{D2O_fraction:.2f}D2O1',f'{1-D2O_fraction:.2f}H2O1'],density)[2]
        
        # determine exact matching condition
        mix = js.dA(np.c_[c,f].T)
        frac = mix.interp(contrast.d2oh2o)[0]
        contrast.matchfraction = frac
        contrast.D2O_fraction=D2O_fraction
        contrast.temperatureC = TT
        contrast.le = f'T={T:.1f} K; {D2O_fraction*100:.1f}% '
        contrasts.append(contrast)
        
        

In [ ]:
p=js.mplot()
for c, contrast in enumerate(contrasts):
    p.plot(contrast,li=c,sy=[1,0.3,c],legend='$le')
    p.plot([0,1],[contrast.d2oh2o,contrast.d2oh2o],sy=0,li=[1,1,c])

p.Yaxis(label=r'scattering length density / nm**-2')
p.Xaxis(label='d fraction in hd-PEG')
p.Text(y=0.00065,x=0.2,string=r'D\s2\NO scattering lenght density')
p.Text(y=0.00065,x=0.2,string=r'')

p.Legend(charsize=0.8)
p.Title(r'PEG matching ' )

In [ ]:
print('D2O_fraction  temperatureC matchfraction')
for contrast in contrasts:
    print(f'{contrast.D2O_fraction:3.2f}, {contrast.temperatureC:10.1f},{contrast.matchfraction:10.3f}')
